# 상태유지 스택 순환신경망 모델

In [1]:
import pandas as pd
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
def create_dataset(signal_data, look_back=1):
    dataX, dataY=[], []
    for i in range(len(signal_data)-look_back):
        dataX.append(signal_data[i:(i+look_back),0])
        dataY.append(signal_data[i+look_back,0])
    return np.array(dataX), np.array(dataY)

In [3]:
class CustomHistory(keras.callbacks.Callback):
    def init(self):
        self.train_loss=[]
        self.val_loss=[]
    
    def on_epoch_end(self, batch, logs={}):
        self.train_loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))
    

In [4]:
train_ = pd.read_csv('../data/train.csv',index_col=0)
test_ = pd. read_csv('../data/test.csv',index_col=0)
sample_submission = pd.read_csv('../data/sample_submission.csv', index_col=0)

In [18]:
look_back=5


In [19]:
signal_data=train_['Y00'][0:4320,None]

In [20]:
signal_data.shape

(4320, 1)

In [21]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#scaler=MinMaxScaler(feature_range=(0,1))
scaler=StandardScaler()
signal_data=scaler.fit_transform(signal_data)


In [22]:
length=len(signal_data)


In [23]:
len1=int(0.8*length)
len2=int(0.1*length)

In [24]:
train = signal_data[:len1]
val = signal_data[len1:len1+len2]
test = signal_data[len1+len2:]

In [25]:
x_train, y_train = create_dataset(train, look_back)
x_val, y_val = create_dataset(val, look_back)
x_test, y_test = create_dataset(test, look_back)

In [26]:
x_train=np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_val=np.reshape(x_val, (x_val.shape[0], x_val.shape[1], 1))
x_test=np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [27]:
model= Sequential()

for i in range(2):
    model.add(LSTM(32, input_shape=( look_back,1) , return_sequences=True, activation='tanh'))
    model.add(Dropout(0.1))

model.add(LSTM(32, activation='tanh'))
model.add(Dropout(0.1))
model.add(Dense(1))

In [28]:
# model= Sequential()
# for i in range(1):
#     model.add(LSTM(32, batch_input_shape=(1, look_back,1) ,stateful=True, return_sequences=True))
#     model.add(Dropout(0.3))

# model.add(LSTM(32,  batch_input_shape=(1, look_back,1) , stateful=True))
# model.add(Dropout(0.3))
# model.add(Dense(1))
# model.summary()

In [29]:
model.compile(loss='MSE',optimizer='adam')
custom_hist=CustomHistory()
custom_hist.init()

In [ ]:
model.fit( x_train, y_train, epochs=100, batch_size=1, shuffle=False, callbacks=[custom_hist], validation_data=(x_val, y_val))

Train on 3451 samples, validate on 427 samples
Epoch 1/100
3451/3451 [==============================] - 35s 10ms/step - loss: 0.1574 - val_loss: 0.0407
Epoch 2/100
3451/3451 [==============================] - 33s 10ms/step - loss: 0.0978 - val_loss: 0.0189
Epoch 3/100
3451/3451 [==============================] - 33s 10ms/step - loss: 0.0519 - val_loss: 0.0225
Epoch 4/100
3451/3451 [==============================] - 33s 10ms/step - loss: 0.0436 - val_loss: 0.0183
Epoch 5/100
3451/3451 [==============================] - 33s 10ms/step - loss: 0.0401 - val_loss: 0.0169
Epoch 6/100
3451/3451 [==============================] - 33s 10ms/step - loss: 0.0376 - val_loss: 0.0099
Epoch 7/100
3451/3451 [==============================] - 33s 10ms/step - loss: 0.0329 - val_loss: 0.0151
Epoch 8/100
1669/3451 [=============>................] - ETA: 16s - loss: 0.0388

In [ ]:
# for i in range(100):
#     print(i)
#     model.fit( x_train, y_train, epochs=1, batch_size=1, shuffle=False, callbacks=[custom_hist], validation_data=(x_val, y_val))
#     model.reset_states()

In [ ]:
trainScore=model.evaluate(x_train, y_train, verbose=0,batch_size=1)
model.reset_states()
print('Train Score:', trainScore)

ValScore=model.evaluate(x_val, y_val, verbose=0,batch_size=1)
model.reset_states()
print('Validation Score:', ValScore)

testScore=model.evaluate(x_test, y_test, verbose=0,batch_size=1)
model.reset_states()
print('Test Score:', testScore)


In [ ]:
look_ahead=len(train_['Y00'])-len(train_['Y00'][0:4320])

In [ ]:
len(y_test)

In [ ]:
look_ahead=432+look_back

xhat=x_test[0]
predictions=np.zeros((look_ahead,1))

for i in range(look_ahead):
    prediction=model.predict(np.array([xhat]), batch_size=1)
    predictions[i]=prediction
    xhat=np.vstack([xhat[1:,],prediction])

In [ ]:
# plt.figure(figsize=(12,5))
# plt.plot(np.arange(look_ahead),predictions,'r',label='prediction')
# plt.plot(np.arange(look_ahead),y_test[:412],label='test function')
# plt.legend()
# plt.show()


In [ ]:
le1=np.arange(len(predictions))
le2=np.arange(len(y_test))

prd=scaler.inverse_transform(predictions)
prs=scaler.inverse_transform(y_test)


In [ ]:

plt.plot(le1,prd,'r',label='prediction')
plt.plot(le2,prs,label='test function')